In [84]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup

In [57]:
# get total number of active power outages in Nova Scotia
def outages_all():
    url = "http://outagemap.nspower.ca/external/default.html"
    options = Options()
    options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return int([i.get_text().replace(",","") for i in soup.find_all("div","total-customers-affected-value")][0])

In [184]:
# returns a dictionary of regions and their number o
def outages_combined():
    url = "http://outagemap.nspower.ca/external/default.html"
    options = Options()
    options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)

    x1 = driver.find_element_by_xpath('//*[@id="menu-content"]/a[2]')
    x1.click()

    x2 = driver.find_element_by_xpath('//*[@id="view-summary-serviceArea"]')
    x2.click()

    driver.implicitly_wait(10)

    outs = {}
    #second last index: increase by 1 up to 34 
    count = 4
    for i in range(31):
        county = driver.find_element_by_xpath(f'//*[@id="report-panel-serviceArea-table"]/tbody/tr[{count}]/td[1]')
        outages = driver.find_element_by_xpath(f'//*[@id="report-panel-serviceArea-table"]/tbody/tr[{count}]/td[2]')
        outs[county.text] = outages.text
        count += 1

    del outs['']

    al = {}
    for k,v in outs.items():
        try:
            a = v.replace(',','')
            al[k] = int(a)
        except:
            al[k] = 5
    al['ALL'] = sum([v for k,v in al.items()])
    
    return al

In [185]:
outages()

{'ALL': 16131,
 'ANTIGONISH': 240,
 'BRIDGEWATER': 3275,
 'CHESTER': 9,
 'CHETICAMP': 139,
 'DARTMOUTH': 1980,
 'HALIFAX': 7169,
 'LIVERPOOL': 46,
 'MABOU': 668,
 'PORT HAWKESBURY': 25,
 'RIVER BOURGEOIS': 1850,
 'SACKVILLE': 51,
 'SHEET HARBOUR': 160,
 "ST. MARGARET'S BAY": 158,
 'STELLARTON': 308,
 'SYDNEY': 38,
 'TRURO': 5,
 'UPPER MUSQUODOBOIT': 5,
 'WINDSOR': 5}